# Importing Libraries

In [ ]:
!pip install hazm

In [ ]:
!pip install --upgrade gensim

In [ ]:
import os
import json
import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pickle

from collections import Counter

# import gensim.models.keyedvectors as word2vec
# from gensim.models import KeyedVectors

# import hazm
# from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

from keras.layers import Dense, Input, LSTM, GRU, Embedding, Flatten, Dropout, Activation
from keras.layers import Bidirectional#, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import Sequential#, Model
# from keras import initializers, regularizers, constraints, optimizers, layers

from sklearn.metrics import confusion_matrix

import gensim
from gensim.corpora.dictionary import Dictionary
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE

**Preprocessing:**

In [ ]:
df = pd.read_csv('D:/Construction Management/Thesis/Data/contracts/Risky_Clauses/Dataset.csv')

In [ ]:
df.head()

In [ ]:
df['risk'].value_counts()

**Split Data to X and y:**

In [ ]:
X = df['clause']
y = df['risk']

**Tokenization:**

**With hazm word_tokenize() method:**

In [ ]:
tokenizer = hazm.word_tokenize

In [ ]:
X_tokenized = []

for clause in X:
  X_tokenized.append(tokenizer(clause))

In [ ]:
X_tokenized

In [ ]:
number_of_tokens = 0

for clause in X_tokenized:
    for token in clause:
        number_of_tokens += 1

print(number_of_tokens)

**With Keras text_to_word_sequence() method:**

In [ ]:
X_tokenized = []

for clause in X:
  X_tokenized.append(text_to_word_sequence(clause))

In [ ]:
X_tokenized

In [ ]:
number_of_tokens = 0

for clause in X_tokenized:
    for token in clause:
        number_of_tokens += 1

print(number_of_tokens)

**With Keras Tokenizer() class:**

In [ ]:
#counting number of distinct words
unique_words = []

for clause in X_tokenized:
  for token in clause:
    if token not in unique_words:
      unique_words.append(token)

In [ ]:
tokenizer = Tokenizer(num_words=len(unique_words)+1, oov_token='<OOV>')
tokenizer.fit_on_texts(list(X))
X_tokenized = tokenizer.texts_to_sequences(X)

In [ ]:
tokenizer.word_index

In [ ]:
X_tokenized

In [ ]:
reverse_word_index = dict([value, key] for (key, value) in tokenizer.word_index.items())

def decode(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

decode(X_tokenized[3])

In [ ]:
#counting number of all words
count = Counter()

for clause in X_tokenized:
    for word in clause:
        count[word] += 1
count

# for key, value in count.items():
#     if value >= 50:
#         print(key)

**Embedding:**

In [ ]:
#load fasttext pretrained embeddings from laptop 
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip

In [ ]:
import zipfile

target = 'v0.9.2.zip'

handle = zipfile.ZipFile(target)
handle.extractall()
handle.close()

In [ ]:
cd ../fastText-0.9.2

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz

In [ ]:
ft.save_model('cc.fa.300.bin')

In [ ]:
import fasttext.util

def pkl_embeddings(word_embedding, unique_words, dimension):
    ft = fasttext.load_model('E:/fastText/cc.fa.300.bin')
    
    if dimension == 300:
        print('trying to make embeddings of dimension == 300')
        for word in unique_words:
          word_embedding[dimension][word] = ft.get_word_vector(word)
        
        with open(f'C:/Users/Pishtaz/Desktop/Thesis_Code/fasttext_embedding_{dimension}_pretrained.pickle','wb') as f:
                pickle.dump(word_embedding, f) 
        
    elif dimension == 200:
        print('trying to make embeddings of dimension == 300')

        fasttext.util.reduce_model(ft, 200)
        
        for word in unique_words:
            word_embedding[dimension][word] = ft.get_word_vector(word)
            
        with open(f'C:/Users/Pishtaz/Desktop/Thesis_Code/fasttext_embedding_{dimension}_pretrained.pickle','wb') as f:
            pickle.dump(word_embedding, f)
        
    elif dimension == 100:
        print('trying to make embeddings of dimension == 300')

        fasttext.util.reduce_model(ft, 100)
            
        for word in unique_words:
            word_embedding[dimension][word] = ft.get_word_vector(word)

        with open(f'C:/Users/Pishtaz/Desktop/Thesis_Code/fasttext_embedding_{dimension}_pretrained.pickle','wb') as f:
            pickle.dump(word_embedding, f)

In [ ]:
import fasttext.util
ft = fasttext.load_model('E:/fastText/cc.fa.300.bin')
print('loaded persian fastText')

word_embedding = {300:{}, 200:{}, 100:{}}

fasttext.util.reduce_model(ft, 100)
print('reduced embedding dimension')

for word in unique_words:
  word_embedding[100][word] = ft.get_word_vector(word)

print('loaded word embeddings')

with open(f'C:/Users/Pishtaz/Desktop/Thesis_Code/fasttext_embedding_{100}_pretrained.pickle','wb') as f:
        pickle.dump(word_embedding, f)

In [ ]:
word_embedding = {300:{}, 200:{}, 100:{}}

pkl_embeddings(word_embedding, unique_words, 300)
pkl_embeddings(word_embedding, unique_words, 200)
pkl_embeddings(word_embedding, unique_words, 100)

In [ ]:
def load_embedding(directory, dimension):
    word_embedding = {300:{}, 200:{}, 100:{}}
    
    with open(os.path.join(directory,f'fasttext_embedding_{dimension}_pretrained.pickle'),'rb') as f:
        word_embedding[dimension] = pickle.load(f)
        
    return word_embedding[dimension]

In [ ]:
directory = 'C:/Users/Pishtaz/Desktop/Thesis_Code/embedding'
emb_dimension = 300

word_embedding = load_embedding(directory, emb_dimension)

**Training fastText on our Corpus:**

In [ ]:
ft_unsupervised = fasttext.train_unsupervised('/content/drive/MyDrive/Colab Notebooks/6. Thesis/fastText-UnsupervisedTraining.txt',
                                              'cbow', minn=3, maxn=6, dim=300, epoch=5, lr=0.001, thread=4)

In [ ]:
len(ft_unsupervised.words)

**Removing Stopwords:**

In [ ]:
with open("persian_stopwords.json",'r',encoding = "utf-8-sig") as f:
    persian_sw = json.load(f)

In [ ]:
def remove_stop_words(X_tokenized, stop_words):

    for sw in stop_words:
        for sentence in X_tokenized:
            try:
                while True:
                    sentence.remove(sw)
            except ValueError:
                pass

In [ ]:
remove_stop_words(X_tokenized, persian_sw)

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(X_tokenized)

In [ ]:
tokenizer.word_index

In [ ]:
# integer encode the documents
encoded_docs = tokenizer.texts_to_sequences(X_tokenized)
print(encoded_docs)

In [ ]:
len(encoded_docs[1])

**Padding:**

In [ ]:
totalNumWords = [len(line) for line in X_tokenized]
plt.hist(totalNumWords,bins = np.arange(0,120,1))
plt.show()

In [ ]:
max(totalNumWords)

In [ ]:
max_length = 50

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post', truncating='post')
print(padded_docs)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
embedding_matrix = np.zeros((vocab_size, emb_dimension))
embedding_matrix.shape

In [ ]:
tokenizer.word_index.items()

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, emb_dimension))

for word, index in tokenizer.word_index.items():
    #word is the key and i is the value of tokenizer.word_index.items() dictionary
    embedding_vector = word_embedding[emb_dimension].get(word)
    
    if index < vocab_size:
        if embedding_vector is not None:
                #words not found in embedding index will be all-zeros
                embedding_matrix[index] = embedding_vector

**SMOTE:**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y,
                                                    test_size=0.2, stratify=y)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority', k_neighbors=7)
X_sm, y_sm = smote.fit_resample(X_train, y_train)
count = Counter(y_sm)
print(count)

**Training and testing various neural networks :**

**Option1: LSTM:**

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_matrix], trainable = False))

model.add(LSTM(64, dropout = 0.5, recurrent_dropout = 0.25, return_sequences = True))
model.add(LSTM(32, dropout = 0.5, recurrent_dropout = 0.25, return_sequences = True))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
start = time.time()
hist = model.fit(X_train, y_train, epochs = 30,
                batch_size = 32, validation_split=0.2)
finish = time.time()

print(finish - start)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,y_test,batch_size=32,verbose=2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model_name = 'LSTM'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res.csv', index=0)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],))

In [ ]:
model_name = 'LSTM'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_model = pd.DataFrame(columns=['accuracy', 'val_accuracy', 'loss', 'val_loss'])

for col in df_log_model.columns:
    df_log_model[col] = hist.history[col]
    
df_log_model.to_csv(base_dir+f'{model_name}-reports-model.csv', index=0)

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res-round.csv', index=0)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

**Option2: BiLSTM:**

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_matrix], trainable = False))
model.add(Bidirectional(LSTM(64, return_sequences=True,
                             name='bilstm_1',dropout=0.5,recurrent_dropout=0.1)))
model.add(LSTM(16, dropout = 0.5, recurrent_dropout = 0.25, name='lstm_1', return_sequences = True))

model.add(Dense(32, activation = 'relu', name = "dense_1"))
model.add(Dropout(0.5))
model.add(Dense(16, activation = 'relu', name = "dense_2"))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid', name = "output"))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, epochs = 30,
                batch_size = 32, validation_split = 0.2)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,y_test,batch_size=32,verbose=2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model_name = 'BiLSTM'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res.csv', index=0)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],))

In [ ]:
model_name = 'BiLSTM'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_model = pd.DataFrame(columns=['accuracy', 'val_accuracy', 'loss', 'val_loss'])

for col in df_log_model.columns:
    df_log_model[col] = hist.history[col]
    
df_log_model.to_csv(base_dir+f'{model_name}-reports-model.csv', index=0)

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res-round.csv', index=0)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

**Option3: FFNN:**

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_matrix], trainable = False))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(8, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, epochs = 30,
                batch_size = 32, validation_split=0.2)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,y_test,batch_size=32,verbose=2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model_name = 'FFNN'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res.csv', index=0)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],))

In [ ]:
model_name = 'FFNN'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_model = pd.DataFrame(columns=['accuracy', 'val_accuracy', 'loss', 'val_loss'])

for col in df_log_model.columns:
    df_log_model[col] = hist.history[col]
    
df_log_model.to_csv(base_dir+f'{model_name}-reports-model.csv', index=0)

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res-round.csv', index=0)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

**Option4: GRU:**

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_matrix], trainable = False))

model.add(GRU(64, dropout = 0.5, recurrent_dropout = 0.25, return_sequences = True, name="gru_1"))
model.add(GRU(32, dropout = 0.5, recurrent_dropout = 0.25, return_sequences = True, name="gru_2"))

model.add(Dense(32, activation = 'relu', name="dense_1"))
model.add(Dropout(0.5))

model.add(Dense(16, activation = 'relu', name="dense_2"))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
start = time.time()
hist = model.fit(X_train, y_train, epochs = 30,
                batch_size = 32, validation_split=0.2)
finish = time.time()

print(finish - start)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,y_test,batch_size=32,verbose=2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model_name = 'GRU'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res.csv', index=0)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],))

In [ ]:
model_name = 'GRU'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_model = pd.DataFrame(columns=['accuracy', 'val_accuracy', 'loss', 'val_loss'])

for col in df_log_model.columns:
    df_log_model[col] = hist.history[col]
    
df_log_model.to_csv(base_dir+f'{model_name}-reports-model.csv', index=0)

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res-round.csv', index=0)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

**Option5: CNN:**

In [ ]:
from keras.layers import  Conv1D, MaxPooling1D, BatchNormalization

model = Sequential()
model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_matrix], trainable = False))

model.add(Conv1D(64, kernel_size=4, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(4))
model.add(Dropout(0.5))

model.add(Conv1D(32, kernel_size=8, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(4))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, epochs = 30,
                batch_size = 32, validation_split=0.2)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,y_test,batch_size=32,verbose=2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
model_name = 'CNN'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res.csv', index=0)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],))

In [ ]:
model_name = 'CNN'
base_dir = 'D:/Construction Management/Thesis/Extracted Paper/Figures/'

df_log_model = pd.DataFrame(columns=['accuracy', 'val_accuracy', 'loss', 'val_loss'])

for col in df_log_model.columns:
    df_log_model[col] = hist.history[col]
    
df_log_model.to_csv(base_dir+f'{model_name}-reports-model.csv', index=0)

df_log_res = pd.DataFrame(columns=['y_test', 'y_pred'])    
df_log_res['y_test'] = list(y_test)
df_log_res['y_pred'] = list(np.ravel(y_pred))

df_log_res.to_csv(base_dir+f'{model_name}-reports-res-round.csv', index=0)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

**Word2Vec:**

In [ ]:
def _emb_matrix_wv(index_dict, word_vectors, **params):

    print("creating embedding matrix for word2vec")
    
    vocab_size = len(index_dict) + 1

    embedding_weights = np.zeros((vocab_size, 300))

    for word, index in index_dict.items():
        embedding_weights[index, :] = word_vectors[word]
        
    return vocab_size, embedding_weights

In [ ]:
def _emb_dict_wv(wv_model, **params):

    if wv_model is not None:

        gensim_dict = Dictionary()
        gensim_dict.doc2bow(wv_model.wv.key_to_index.keys(), allow_update=True)

        w2indx = {v: k+1 for k, v in gensim_dict.items()}
        w2vec = {word: wv_model.wv.get_vector(word) for word in w2indx.keys()}
        
        return w2indx, w2vec

    else:
        print('Word2Vec model is not trained yet')

In [ ]:
def train_word2vec(X_tokenized, **params):
    print("training word2vec model")

    wv_model = gensim.models.word2vec.Word2Vec(sentences = X_tokenized,
                                            vector_size = 300,
                                            window = 5,
                                            min_count = 3,
                                            sg = 1,
                                            epochs = 5,)

    wv_model.build_vocab(X_tokenized)

    wv_model.train(X_tokenized, total_examples=wv_model.corpus_count, epochs = 5,)
    wv_model.save("C:/Users/Pishtaz/Desktop/other/Thesis/Risky-Clauses-NLP/embedding-jupyter/w2v_model.pkl")

    index_dict, word_vectors = _emb_dict_wv(X = X_tokenized, wv_model = wv_model)

    return index_dict, word_vectors

In [ ]:
def _save_network_info(index_dict,
                        vocab_size,
                        embedding_weights):


    base_dir = "C:/Users/Pishtaz/Desktop/other/Thesis/Risky-Clauses-NLP/embedding-jupyter/"
    with open(base_dir + 'index_dict.pkl', 'wb') as handle:
        pickle.dump(index_dict, handle)

    with open(base_dir + 'vocab_size.pkl', 'wb') as handle:
        pickle.dump(vocab_size, handle)

    with open(base_dir + 'embedding_weights.pkl', 'wb') as handle:
        pickle.dump(embedding_weights, handle)

In [ ]:
def create_word2vec_embedding(X_tokenized,**params):

    index_dict, word_vectors = train_word2vec(X_tokenized, **params)
    vocab_size, embedding_weights = _emb_matrix_wv(index_dict,
                                                word_vectors,
                                                **params)

    _save_network_info(index_dict, vocab_size, embedding_weights)
    
    return index_dict, vocab_size, embedding_weights

In [ ]:
index_dict, vocab_size, embedding_weights = create_word2vec_embedding(X_tokenized)

In [ ]:
with open("persian_stopwords.json",'r',encoding = "utf-8-sig") as f:
    persian_sw = json.load(f)
    
def remove_stop_words(X_tokenized, stop_words):

    for sw in stop_words:
        for sentence in X_tokenized:
            try:
                while True:
                    sentence.remove(sw)
            except ValueError:
                pass
            
remove_stop_words(X_tokenized, persian_sw)

In [ ]:
def parsing(X_tokenized, w2indx):

    parsed = []
    for sentence in X_tokenized:
        indexed_sentence = []

        for token in sentence:
            try:
                indexed_sentence.append(w2indx[token])
            except:
                indexed_sentence.append(0)

        parsed.append(indexed_sentence)

    return parsed

In [ ]:
def padding(parsed, **params):

    print("padding the parsed sentences...")
    
    X = pad_sequences(parsed, maxlen = 50,
                              padding = 'post',
                              truncating = 'post')

    return X

In [ ]:
def oversample(X_train, Y_train, **params):
    k_nbrs_overs = params.get("k_nbrs_overs")
    
    smote = SMOTE(sampling_strategy = 'minority', k_neighbors = 5)

    X_train, Y_train = smote.fit_resample(X_train, Y_train)

    return X_train, Y_train

In [ ]:
def train_test_split_(X, Y, **params):
    print("splitting data to train and test...")

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,
         shuffle = True, stratify = Y)

    Y_train, Y_test = np.array(Y_train), np.array(Y_test)

    return X_train, X_test, Y_train, Y_test

In [ ]:
def parse_pad_x_train_test(X_tokenized, Y, index_dict, vocab_size, embedding_weights, **params):

    print ("Trying to parse_pad_and_save_x_train_test...")

    X_tokenized = parsing(X_tokenized, index_dict, **params)
    X = padding(X_tokenized, **params)

    X_train, X_test, Y_train, Y_test = train_test_split_(X, Y, **params)
#     X_train, Y_train = oversample(X_train, Y_train, **params)
    
    return X_train, Y_train, X_test, Y_test

In [ ]:
X_train, Y_train, X_test, Y_test = parse_pad_x_train_test(X_tokenized, y, index_dict, vocab_size, embedding_weights)

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, emb_dimension, input_length = max_length,
                    weights = [embedding_weights], trainable = False))
model.add(Bidirectional(LSTM(32, return_sequences=True,
                             name='bilstm_1',dropout=0.5,recurrent_dropout=0.1)))
model.add(LSTM(16, dropout = 0.5, recurrent_dropout = 0.25, name='lstm_1', return_sequences = True))

model.add(Dense(32, activation = 'relu', name = "dense_1"))
model.add(Dropout(0.5))
model.add(Dense(16, activation = 'relu', name = "dense_2"))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid', name = "output"))
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train, epochs = 30,
                batch_size = 32, validation_split = 0.2)

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

In [ ]:
# Visualize the Model
# Plot training & validation accuracy values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
model.evaluate(X_test,Y_test,batch_size=32,verbose=2)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred = np.round(Y_pred)

In [ ]:
Y_pred = np.reshape(Y_pred,(Y_pred.shape[0],))

In [ ]:
print(classification_report(Y_test,Y_pred, digits=4))